In [13]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Master Machine Learning/Way to building a great machine learning model/dataset/train.csv")
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## Create new features from feature

#### Create avegarate total bill



In [14]:
data[["Deck","Side","Number"]] = (
    data["Cabin"].str.split("/", expand=True)
)
data[["Deck","Side","Number"]].head()

,Deck,Side,Number
0,B,0,P
1,F,0,S
2,A,0,S
3,A,0,S
4,F,1,S


### Join simple features into a composed feature if you had reason to believe there was some interaction in the combination:

In [15]:
data["Total_bill"] = data["RoomService"] + data["FoodCourt"]+ data["ShoppingMall"]+ data["Spa"] + data["VRDeck"]
#data["Total_bill"].head()
data["Total_bill"].head()

0        0.0
1      736.0
2    10383.0
3     5176.0
4     1091.0
Name: Total_bill, dtype: float64

#### Group Transforms
>+ Tính toán tần suất mà các trạng thái xảy ra trong tập dữ liệu


In [16]:
data["Total_bill_Averagate"] = (
    data.groupby("Name")["Total_bill"] ## for each name select Total_bill
    .transform("mean") # compute its mean
) 
data["Total_bill_Averagate"].head()

0        0.0
1      736.0
2    10383.0
3     5176.0
4     1091.0
Name: Total_bill_Averagate, dtype: float64

In [17]:
data["HomePlanet_Freq"] = (
    data.groupby("Name")
    ["HomePlanet"]
    .transform("count")
    /data.Name.count()
)
data[["Name", "HomePlanet_Freq"]].head()

,Name,HomePlanet_Freq
0,Maham Ofracculy,0.000118
1,Juanna Vines,0.000118
2,Altark Susent,0.000118
3,Solam Susent,0.000118
4,Willy Santantines,0.000118


### Tips on Creating Features
>+ Hầu hết các mô hình đều khó học được các tỷ lệ. Kết hợp tỷ lệ thường dẫn đến một số hiệu suất dễ dàng đạt được
>+ Mô hình tuyến tính học tổng và chênh lệch một cách tự nhiên, nhưng không thể học bất cứ điều gì phức tạp hơn
>+ Các mô hình tuyến tính và mạng nơ ron thường hoạt động tốt hơn với các tính năng chuẩn hóa. Lưới thần kinh đặc biệt cần các tính năng được chia tỷ lệ đến giá trị không quá xa 0. Các mô hình dựa trên cây (như rừng ngẫu nhiên và XGBoost) đôi khi có thể được lợi từ việc chuẩn hóa, nhưng thường ít hơn nhiều.
>+ Mô hình "Tree" có thể học cách tính gần đúng hầu hết mọi sự kết hợp của các tính năng, nhưng khi một sự kết hợp đặc biệt quan trọng, chúng vẫn có thể hưởng lợi từ việc tạo nó một cách rõ ràng, đặc biệt khi dữ liệu bị hạn chế.

#### Create function evalue score of model

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

def score_dataset(X, y, model=LogisticRegression()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score